## Klasifikasi

Klasifikasi yang dilakukan adalah dengan pendekatan Lexicon dengan library dari Bing Liu yang di traslate dalam bahasa Indonesia

In [1]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv("../../data/dataset/datasetpraproses_nostop.csv", sep=';')

In [3]:
dataset

,Tweet,label,Tweet_Parsed
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif,admin normal ke kalau baru pakai beberapa ...
1,Kmera iphone 11 mmeng beda,kamera positif,kamera memang beda
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif,kamera depannya jelek aku liat liat para p...
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif,meluncurkan dengan peningkatan daya tahan ba...
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...","spesifikasi positif,harga positif",tapi rugi kalau kamu beli baru sekarang ...
...,...,...,...
743,ga keren banget masa charger bawaannya belum f...,baterai negatif,tidak keren sekali masa charger bawaannya belu...
744,"yaelah belum fast charging, lama nih kalo ngec...",baterai negatif,yaelah belum cepat charger lama kalau charge ...
745,butuh waku lama buat ngecharge kalo pake charg...,baterai negatif,butuh waku lama buat charge kalau menggunakan ...
746,udah mahal begini masa chargernya blm fast cha...,baterai negatif,udah mahal begini masa chargernya belum cepat ...


In [4]:
lexi = pd.read_csv("../../data/lexicon/lexiinset_final.csv", sep=";")
lexi

,word,weight
0,hai,3
1,merekam,2
2,ekstensif,3
3,paripurna,1
4,detail,2
...,...,...
10213,kantong kering,-4
10214,penggaruk,-3
10215,senewen,-4
10216,menetapkan,-5


In [5]:
def klasifikasi(text, lexicon):
    weight = 0
    for i in range(5):
        n = i+1
        n_grams = [str(text).split(' ')[i-(n-1):i+1] for i in range(n-1,len(str(text).split(' ')))]
        for j in n_grams:
            text_grams = ' '.join(j)
            for k in lexicon:
                if text_grams==k[0]:
                    weight = weight+k[1]
#     weight = ' '.join(map(str, text))
    return weight

# aspect extraction

In [6]:
dataset = pd.read_csv("../../data/dataset/datasetpraproses_nostop.csv", sep=';')

In [7]:
subaspect = pd.read_csv("../../data/dataset/newsubaspect.csv", sep=";")

In [8]:
subaspect

,aspect,subaspect
0,baterai,mah
1,baterai,kapasitas
2,baterai,daya
3,baterai,charge
4,baterai,charger
...,...,...
93,general,speakernya
94,kamera,angle
95,spesifikasi,speknya
96,spesifikasi,spesifikasi


In [9]:
lexi = pd.read_csv("../../data/lexicon/lexiinset_final.csv", sep=";")

In [10]:
dataset.head()

,Tweet,label,Tweet_Parsed
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif,admin normal ke kalau baru pakai beberapa ...
1,Kmera iphone 11 mmeng beda,kamera positif,kamera memang beda
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif,kamera depannya jelek aku liat liat para p...
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif,meluncurkan dengan peningkatan daya tahan ba...
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...","spesifikasi positif,harga positif",tapi rugi kalau kamu beli baru sekarang ...


In [11]:
label = []
label_score = []
for i in dataset['Tweet_Parsed'].values:
    split = i.split(" ")
    n = len(split)
    start = 0
    j = start
    sentiment = []
    sentiment_score = []
    aspect_temp = []
    zero_score = []
    while start<n and j<n:
        score = 0
        if split[j] in subaspect['subaspect'].values:
            aspect = subaspect[subaspect['subaspect']==split[j]]['aspect'].values[0]
            if j==start:
                k = j+1
                while k<n:
                    if split[k] in subaspect['subaspect'].values:
                        break
                    k = k+1
                opinion = " ".join(split[j:k])
                score = klasifikasi(opinion,lexi.values)
                start = k
            elif j>start:
                k = j
                opinion = " ".join(split[start:k+1])
                score = klasifikasi(opinion,lexi.values)
                start = j+1
                if score==0 and j!=n-1:
                    k = j+1
                    while k<n:
                        if split[k] in subaspect['subaspect'].values:
                            break
                        k = k+1
                    opinion = " ".join(split[j:k])
                    score = klasifikasi(opinion,lexi.values)
                    start = k
            lab_score = "positif" if score>=0 else "negatif"
            if score != 0:
                if aspect not in aspect_temp:
                    sentiment.append(" ".join([aspect,lab_score]))
                    sentiment_score.append(" ".join([aspect,str(score)]))
                aspect_temp.append(aspect)
            else:
                if aspect not in zero_score:
                    zero_score.append(aspect)
            j = start
        else:
            j = j+1
    if len(sentiment) == 0 and len(zero_score) == 0:
        aspect = "general"
        score = klasifikasi(i,lexi.values)
        lab_score = "positif" if score >= 0 else "negatif"
        sentiment.append(" ".join([aspect,lab_score]))
        sentiment_score.append(" ".join([aspect,str(score)]))
    if len(zero_score) > 0:
        for l in zero_score:
            if l not in aspect_temp:
                score = 0
                lab_score = "positif"
                sentiment.append(" ".join([l,lab_score]))
                sentiment_score.append(" ".join([l,str(score)]))
    label.append(",".join(sentiment))
    label_score.append(",".join(sentiment_score))

In [12]:
# label_score
len(label_score)

748

In [13]:
len(label)

748

In [14]:
dataset['Prediksi']=label
dataset['Prediksi_Score'] = label_score

In [15]:
dataset

,Tweet,label,Tweet_Parsed,Prediksi,Prediksi_Score
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif,admin normal ke kalau baru pakai beberapa ...,baterai negatif,baterai -4
1,Kmera iphone 11 mmeng beda,kamera positif,kamera memang beda,kamera positif,kamera 0
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif,kamera depannya jelek aku liat liat para p...,kamera negatif,kamera -3
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif,meluncurkan dengan peningkatan daya tahan ba...,baterai positif,baterai 3
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...","spesifikasi positif,harga positif",tapi rugi kalau kamu beli baru sekarang ...,"spesifikasi negatif,harga negatif","spesifikasi -9,harga -1"
...,...,...,...,...,...
743,ga keren banget masa charger bawaannya belum f...,baterai negatif,tidak keren sekali masa charger bawaannya belu...,baterai negatif,baterai -6
744,"yaelah belum fast charging, lama nih kalo ngec...",baterai negatif,yaelah belum cepat charger lama kalau charge ...,baterai negatif,baterai -5
745,butuh waku lama buat ngecharge kalo pake charg...,baterai negatif,butuh waku lama buat charge kalau menggunakan ...,baterai negatif,baterai -8
746,udah mahal begini masa chargernya blm fast cha...,baterai negatif,udah mahal begini masa chargernya belum cepat ...,"harga positif,baterai negatif","harga 1,baterai -1"


In [16]:
dataset.to_csv("../../data/prediksi/prediksi_inset_nostop.csv", sep=";", index=False)